# Small-Model Training

Train a small linear model over fixed embeddings, using a curated set of training data.



# Imports and Configuration.

In [ ]:
#@title Imports. { vertical-output: true }

# Global imports
import collections
import json
from ml_collections import config_dict
import numpy as np
import tensorflow as tf
from etils import epath
import matplotlib.pyplot as plt

from chirp.inference import colab_utils
colab_utils.initialize(use_tf_gpu=True, disable_warnings=True)

# Chirp imports
from chirp.models import metrics
from chirp.inference import tf_examples
from chirp.projects.bootstrap import bootstrap
from chirp.projects.bootstrap import display
from chirp.projects.bootstrap import search
from chirp.projects.multicluster import classify
from chirp.projects.multicluster import data_lib


In [ ]:
#@title Configure data and model locations. { vertical-output: true }

# Path containing TFRecords of unlabeled embeddings.
# We will load the model which was used to compute the embeddings automatically.
embeddings_path = '/tmp/embeddings'  #@param

# Path to the labeled wav data.
# Should be in 'folder-of-folders' format - a folder with sub-folders for
# each class of interest.
# Audio in sub-folders should be wav files.
# Audio should ideally be 5s audio clips, but the system is quite forgiving.
labeled_data_path = ''  #@param


In [ ]:
#@title Load the model. { vertical-output: true }

# Get relevant info from the embedding configuration.
embeddings_path = epath.Path(embeddings_path)
with (embeddings_path / 'config.json').open() as f:
  embedding_config = config_dict.ConfigDict(json.loads(f.read()))
embeddings_glob = embeddings_path / 'embeddings-*'
embedding_hop_size_s = embedding_config.embed_fn_config.model_config.hop_size_s

config = bootstrap.BootstrapConfig.load_from_embedding_config(
    embeddings_path=embeddings_path,
    annotated_path=labeled_data_path)
project_state = bootstrap.BootstrapState(config)
embedding_model = project_state.embedding_model

# Supervised Learning.

In [ ]:
#@title Load+Embed the Labeled Dataset. { vertical-output: true }

# Time-pooling strategy for examples longer than the model's window size.
time_pooling = 'mean'  #@param

merged = data_lib.MergedDataset(config.annotated_path,
                                embedding_model,
                                time_pooling=time_pooling)

# Label distribution
lbl_counts = np.sum(merged.data['label_hot'], axis=0)
print('num classes :', (lbl_counts > 0).sum())
print('mean ex / class :', lbl_counts.sum() / (lbl_counts > 0).sum())
print('min ex / class :', (lbl_counts + (lbl_counts == 0) * 1e6).min())


In [ ]:
#@title Train linear model over embeddings. { vertical-output: true }

# Number of random training examples to choose form each class.
example_per_class = 128  #@param

# Number of random re-trainings. Allows judging model stability.
num_seeds = 1  #@param

# Classifier training hyperparams.
# These should be good defaults.
batch_size = 32
num_epochs = 128
num_hiddens = -1
learning_rate = 1e-3

metrics = collections.defaultdict(list)
for seed in range(num_seeds):
  if num_hiddens > 0:
    model = classify.get_two_layer_model(
        num_hiddens, merged.embedding_dim, merged.num_classes)
  else:
    model = classify.get_linear_model(
        merged.embedding_dim, merged.num_classes)
  run_metrics = classify.train_embedding_model(
      model, merged, example_per_class, num_epochs, seed, batch_size, learning_rate)
  metrics['acc'].append(run_metrics.top1_accuracy)
  metrics['auc_roc'].append(run_metrics.auc_roc)
  metrics['cmap'].append(run_metrics.cmap_value)
  metrics['maps'].append(run_metrics.class_maps)
  metrics['recall'].append(run_metrics.recall)
mean_acc = np.mean(metrics['acc'])
mean_auc_roc = np.mean(metrics['auc_roc'])
mean_cmap = np.mean(metrics['cmap'])
mean_recall = np.mean(metrics['recall'])
print(f'{example_per_class:d},  acc:{mean_acc:5.2f},  '
      f'auc_roc:{mean_auc_roc:5.2f},  cmap:{mean_cmap:5.2f},  '
      f'recall:{mean_recall:5.2f}')
for lbl, auc in zip(merged.labels, run_metrics.class_maps):
  if np.isnan(auc):
    continue
  print(f'{lbl:8s}, auc_roc:{auc:5.2f}')

# Evaluation on Unlabeled Data

In [ ]:
#@title Run model on target unlabeled data. { vertical-output: true }

# Choose the target class to work with.
target_class = ''  #@param
# Choose a target logit; will display results close to the target.
target_logit = 2.0  #@param
# Number of results to display.
num_results = 25  #@param

# Create the embeddings dataset.
embeddings_ds = tf_examples.create_embeddings_dataset(
    embeddings_path, file_glob='embeddings-*')
target_class_idx = merged.labels.index(target_class)
results, all_logits = search.classifer_search_embeddings_parallel(
    embeddings_ds, model, target_class_idx, hop_size_s=embedding_hop_size_s,
    target_logit=target_logit, top_k=num_results
)

# Plot the histogram of logits.
_, ys, _ = plt.hist(all_logits, bins=128, density=True)
plt.xlabel(f'{target_class} logit')
plt.ylabel('density')
# plt.yscale('log')
plt.plot([target_logit, target_logit], [0.0, np.max(ys)], 'r:')
plt.show()


In [ ]:
#@title Display results for the target label. { vertical-output: true }

display_labels = merged.labels

extra_labels = []  #@param
for label in extra_labels:
  if label not in merged.labels:
    display_labels += (label,)
if 'unknown' not in merged.labels:
  display_labels += ('unknown',)

display.display_search_results(
    results, embedding_model.sample_rate,
    project_state.source_map,
    checkbox_labels=display_labels,
    max_workers=5)

In [ ]:
#@title Add selected results to the labeled data. { vertical-output: true }

results.write_labeled_data(
    config.annotated_path, embedding_model.sample_rate)

In [ ]:
#@title Write classifier inference CSV. { vertical-output: true }

threshold = 1.0  #@param
output_filepath = '/tmp/inference.csv'  #@param

# Create the embeddings dataset.
embeddings_ds = tf_examples.create_embeddings_dataset(
    embeddings_path, file_glob='embeddings-*')

def classify_batch(batch):
  """Classify a batch of embeddings."""
  emb = batch[tf_examples.EMBEDDING]
  emb_shape = tf.shape(emb)
  flat_emb = tf.reshape(emb, [-1, emb_shape[-1]])
  logits = model(flat_emb)
  logits = tf.reshape(
      logits, [emb_shape[0], emb_shape[1], tf.shape(logits)[-1]])
  # Take the maximum logit over channels.
  logits = tf.reduce_max(logits, axis=-2)
  batch['logits'] = logits
  return batch

inference_ds = tf_examples.create_embeddings_dataset(
    embeddings_path, file_glob='embeddings-*')
inference_ds = inference_ds.map(
    classify_batch, num_parallel_calls=tf.data.AUTOTUNE
)

with open(output_filepath, 'w') as f:
  # Write column headers.
  headers = ['filename', 'timestamp_s', 'label', 'logit']
  f.write(', '.join(headers) + '\n')
  for ex in inference_ds.as_numpy_iterator():
    for t in range(ex['logits'].shape[0]):
      for i, label in enumerate(merged.class_names):
        if ex['logits'][t, i] > threshold:
          offset = ex['timestamp_s'] + t * config.embedding_hop_size_s
          logit = '{:.2f}'.format(ex['logits'][t, i])
          row = [ex['filename'].decode('utf-8'),
                 '{:.2f}'.format(offset),
                 label, logit]
          f.write(', '.join(row) + '\n')
